In [1]:
# Colab DreamBooth Notebook - Especulativo de Biodiversidad Colombiana (Plug & Play)
# Ejecuta 100% en Google Colab usando tu carpeta de Drive 'Dreambooth_Bio'.

# ==========================
# 0. Montar Google Drive
# ==========================
from google.colab import drive
import os

drive.mount('/content/drive')  # Autoriza acceso a tu Drive
# Define ruta a tu carpeta de imágenes en Drive
dataset_dir = '/content/drive/MyDrive/'
# Verifica que la carpeta exista y contenga imágenes
if not os.path.isdir(dataset_dir):
    raise FileNotFoundError(f"No existe la carpeta: {dataset_dir}")
imgs = [f for f in os.listdir(dataset_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]
print(f"Imágenes encontradas en Drive: {len(imgs)} (recomendado 20-50)")
if len(imgs) < 10:
    raise ValueError("Sube al menos 10 imágenes a Dreambooth_Bio en tu Drive antes de continuar.")

# ==========================

Mounted at /content/drive
Imágenes encontradas en Drive: 10 (recomendado 20-50)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 1. Instalar dependencias
# ==========================
# Actualiza pip
!pip install --upgrade pip

# Instala herramientas de aceleración y acceso a modelos de Hugging Face
!pip install accelerate huggingface_hub

# Instala Diffusers, Transformers y PEFT para DreamBooth
!pip install git+https://github.com/huggingface/diffusers
!pip install transformers
!pip install -U peft

# Asegura instalación de torchvision para Accelerate y timm
!pip install torchvision

# ==========================

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 

In [4]:
# ==========================
# 2. Clonar ejemplo DreamBooth. Clonar ejemplo DreamBooth Clonar ejemplo DreamBooth
# ==========================
!git clone https://github.com/huggingface/diffusers.git
%cd diffusers/examples/dreambooth

# ==========================

Cloning into 'diffusers'...
remote: Enumerating objects: 98051, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 98051 (delta 49), reused 37 (delta 23), pack-reused 97933 (from 3)
Receiving objects: 100% (98051/98051), 73.14 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (72340/72340), done.
/content/diffusers/examples/dreambooth


In [5]:
# ==========================
# 2.5 Verificar directorio de trabajo
# ==========================
# Muestra la ruta actual y lista los archivos/scripts disponibles
print("Directorio actual:")
!pwd
print("Contenido de la carpeta:")
!ls -lah

Directorio actual:
/content/diffusers/examples/dreambooth
Contenido de la carpeta:
total 1.1M
drwxr-xr-x  2 root root 4.0K Jul  8 01:31 .
drwxr-xr-x 24 root root 4.0K Jul  8 01:31 ..
-rw-r--r--  1 root root 1016 Jul  8 01:31 convert_to_imagefolder.py
-rw-r--r--  1 root root  16K Jul  8 01:31 README_flux.md
-rw-r--r--  1 root root 5.8K Jul  8 01:31 README_hidream.md
-rw-r--r--  1 root root 4.8K Jul  8 01:31 README_lumina2.md
-rw-r--r--  1 root root  30K Jul  8 01:31 README.md
-rw-r--r--  1 root root 5.0K Jul  8 01:31 README_sana.md
-rw-r--r--  1 root root  11K Jul  8 01:31 README_sd3.md
-rw-r--r--  1 root root  12K Jul  8 01:31 README_sdxl.md
-rw-r--r--  1 root root   74 Jul  8 01:31 requirements_flax.txt
-rw-r--r--  1 root root  102 Jul  8 01:31 requirements_flux.txt
-rw-r--r--  1 root root  101 Jul  8 01:31 requirements_hidream.txt
-rw-r--r--  1 root root  101 Jul  8 01:31 requirements_sana.txt
-rw-r--r--  1 root root  102 Jul  8 01:31 requirements_sd3.txt
-rw-r--r--  1 root root   87

In [6]:
# ==========================
# 3. Configurar Accelerate
# ==========================
from accelerate.utils import write_basic_config
write_basic_config()

# ==========================

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [7]:
# 4. Login a Hugging Face Hub
# ==========================
from huggingface_hub import login
print("Pega tu token de Hugging Face:")
login()

Pega tu token de Hugging Face:


In [8]:
# 5. Parámetros y Fine-Tuning (usando LoRA para ahorro de memoria)
# ==========================
import os
# Configuración de entrenamiento
model_name = "CompVis/stable-diffusion-v1-4"              # Modelo base
instance_prompt = "criatura especulativa colombiana"     # Prompt para tu concepto
resolution = 256                                          # Resolución reducida
train_batch_size = 1                                      # Batch size
max_train_steps = 200                                     # Pasos de entrenamiento reducidos
# Rutas en Drive
dataset_dir = '/content/drive/MyDrive/Dreambooth_Bio'
output_dir = '/content/drive/MyDrive/Dreambooth_Bio_Output'
os.makedirs(output_dir, exist_ok=True)
# Hugging Face Hub
HF_USERNAME = "NOISK8"
HF_MODEL_ID = f"{HF_USERNAME}/dreambooth-bio"

# Construir comando de entrenamiento con LoRA (reduce memoria significativamente)
cmd = (
    f"accelerate launch train_dreambooth_lora.py "
    f"--pretrained_model_name_or_path {model_name} "
    f"--instance_data_dir {dataset_dir} "
    f"--output_dir {output_dir} "
    f"--instance_prompt \"{instance_prompt}\" "
    f"--resolution {resolution} "
    f"--train_batch_size {train_batch_size} "
    f"--max_train_steps {max_train_steps} "
    f"--gradient_checkpointing "
    f"--mixed_precision fp16 "
    f"--push_to_hub "
    f"--hub_model_id {HF_MODEL_ID}"
)
print("Ejecutando fine-tuning LoRA y push a Hub:")
print(cmd)
get_ipython().system(cmd)

Ejecutando fine-tuning LoRA y push a Hub:
accelerate launch train_dreambooth_lora.py --pretrained_model_name_or_path CompVis/stable-diffusion-v1-4 --instance_data_dir /content/drive/MyDrive/Dreambooth_Bio --output_dir /content/drive/MyDrive/Dreambooth_Bio_Output --instance_prompt "criatura especulativa colombiana" --resolution 256 --train_batch_size 1 --max_train_steps 200 --gradient_checkpointing --mixed_precision fp16 --push_to_hub --hub_model_id NOISK8/dreambooth-bio
ipex flag is deprecated, will be removed in Accelerate v1.10. From 2.7.0, PyTorch has all needed optimizations for Intel CPU and XPU.
2025-07-08 01:32:17.940206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751938337.957527    1858 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 

In [9]:
# 6. Inferencia y generación con LoRA
# ==========================
from diffusers import StableDiffusionPipeline
import torch, os

# Carga pipeline base de Stable Diffusion
pipeline = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    safety_checker=None
)

# Carga pesos LoRA entrenados
lora_path = output_dir  # aquí debe estar pytorch_lora_weights.safetensors
pipeline.unet.load_attn_procs(lora_path)

# Mueve pipeline a GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
pipeline.to(device)

# Parámetros de generación
num_images = 4
num_inference_steps = 50
guidance_scale = 7.5

# Generar imágenes
images = pipeline(
    instance_prompt,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    num_images_per_prompt=num_images
).images

# Guardar resultados
generated_paths = []
for idx, img in enumerate(images):
    path = os.path.join(output_dir, f"generated_{idx}.png")
    img.save(path)
    generated_paths.append(path)
    print(f"Guardada: {path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/usr/local/lib/python3.11/dist-packages/diffusers/loaders/unet.py:211: FutureWarning: `load_attn_procs` is deprecated and will be removed in version 0.40.0. Using the `load_attn_procs()` method has been deprecated and will be removed in a future version. Please use `load_lora_adapter()`.
  deprecate("load_attn_procs", "0.40.0", deprecation_messag

  0%|          | 0/50 [00:00<?, ?it/s]

Guardada: /content/drive/MyDrive/Dreambooth_Bio_Output/generated_0.png
Guardada: /content/drive/MyDrive/Dreambooth_Bio_Output/generated_1.png
Guardada: /content/drive/MyDrive/Dreambooth_Bio_Output/generated_2.png
Guardada: /content/drive/MyDrive/Dreambooth_Bio_Output/generated_3.png


In [10]:
# 7. Interfaz de Usuario para Generación de Imágenes
# ==========================
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import io
import base64

# Configuración de la interfaz
style = {'description_width': '150px'}
layout = widgets.Layout(width='80%')

# Crear widgets
prompt_input = widgets.Textarea(
    value='Una criatura mágica de la selva colombiana con alas de mariposa',
    placeholder='Ingresa tu descripción aquí...',
    description='Prompt:',
    disabled=False,
    layout=layout,
    style=style
)

num_images_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=4,
    step=1,
    description='Número de imágenes:',
    disabled=False,
    style=style
)

steps_slider = widgets.IntSlider(
    value=50,
    min=20,
    max=100,
    step=10,
    description='Pasos de inferencia:',
    disabled=False,
    style=style
)

guidance_slider = widgets.FloatSlider(
    value=7.5,
    min=1.0,
    max=15.0,
    step=0.5,
    description='Guía de escala:',
    disabled=False,
    style=style
)

generate_button = widgets.Button(
    description='Generar Imágenes',
    button_style='success',
    tooltip='Haz clic para generar imágenes'
)

output = widgets.Output()

# Función para manejar el clic del botón
def on_generate_button_clicked(b):
    with output:
        clear_output(wait=True)
        print("Generando imágenes, por favor espera...")

        # Obtener valores de los widgets
        prompt = prompt_input.value
        num_images = num_images_slider.value
        num_inference_steps = steps_slider.value
        guidance_scale = guidance_slider.value

        # Generar imágenes
        images = pipeline(
            prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            num_images_per_prompt=num_images
        ).images

        # Mostrar imágenes
        for i, img in enumerate(images):
            display(img)

            # Guardar la imagen
            img_path = f"{output_dir}/generated_custom_{i}.png"
            img.save(img_path)
            print(f"Imagen guardada en: {img_path}")

# Conectar el botón a la función
generate_button.on_click(on_generate_button_clicked)

# Mostrar la interfaz
print("\n" + "="*50)
print("GENERADOR DE CRIATURAS ESPECULATIVAS COLOMBIANAS")
print("="*50 + "\n")
print("Personaliza los parámetros y haz clic en 'Generar Imágenes'")

# Organizar la interfaz
ui = widgets.VBox([
    prompt_input,
    num_images_slider,
    steps_slider,
    guidance_slider,
    generate_button
])

display(ui, output)

# ==========================


GENERADOR DE CRIATURAS ESPECULATIVAS COLOMBIANAS

Personaliza los parámetros y haz clic en 'Generar Imágenes'


Output()